# Utils
> To visualize the dynamics of a NN.

In [ ]:
#| default_exp recurrent.utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from wafer.basics import *

## Data

In [ ]:
#| export
def mk_sequential_MNIST_dls(sz: list[int]=[None, None], # Sizes of train and test set, if `None` then returns all
                            bs: list[int]=[128, 128],   # Batch sizes
                            permute: bool=False,        # Apply a fixed permutation to input sequences
                            device=None):
    mnist = load_dataset("mnist", device=device, trust_remote_code=True)
    train = Dataset.from_dict(mnist['train'][:sz[0]]).with_format('torch') if sz[0] is not None else mnist['train'].with_format('torch')
    test  = Dataset.from_dict(mnist['test'][:sz[1]]).with_format('torch') if sz[1] is not None else mnist['test'].with_format('torch')
    if permute:
        perm = np.random.choice(784, 784, False)
        get_xy = lambda o: ((o['image'].float().reshape(o['image'].shape[0], -1, 1))[:, perm], o['label'])
    else:
        get_xy = lambda o: (o['image'].float().reshape(o['image'].shape[0], -1, 1), o['label'])
    return mk_dls_from_ds([train, test], get_xy, bs=bs)

## Plot

In [ ]:
#| export
def eigen_show(A: np.ndarray|list[list] = [[1,1],[1,1]], figsize=(3,3)):
    "Animate (2D) linear transformation of a matrix."
    assert np.ndim(A) == 2, "Matrix is not 2-by-2."
    fig,ax = plt.subplots(figsize=figsize)
    xs = np.vstack([np.cos((ts:=np.linspace(0,2*np.pi,100))),np.sin(ts)])
    ys = A@xs
    l1, = ax.plot([], [], 'C9-', label='x')
    l2, = ax.plot([], [], 'C3-', label='Ax')
    p1, = ax.plot([], [], 'C9x')
    p2, = ax.plot([], [], 'C3x')
    lb,ub = np.min(ys) - .8, np.max(ys) + .8
    ax.set_xlim(lb, ub); ax.set_ylim(lb, ub)
    ax.set_xlabel(r'$x_1$'); ax.set_ylabel(r'$x_2$')
    ax.axis('equal'); ax.legend(loc=2, fontsize=9)
    def _update(n):
        l1.set_data(xs[0][:n], xs[1][:n])
        l2.set_data(ys[0][:n], ys[1][:n])
        p1.set_data([xs[0][n]], [xs[1][n]])
        p2.set_data([ys[0][n]], [ys[1][n]])
        return (l1,l2)
    anim = animation.FuncAnimation(fig, _update, frames=len(xs[0]), interval=50, blit=True, repeat=False)
    fig.tight_layout(); display(HTML(anim.to_html5_video())); plt.close(fig)

In [ ]:
#| export
def dynshow(f: callable, # Dynamical system (x_new, dx) = f(x)
            ax=None,
            n_pts: int=3,
            n_steps: int=20,
            center: tuple[float]=(0.,0.),
            lims: tuple[float]=(-1.,1.,-1.,1.), # x,y limits
            showfield: bool=False, # To draw the vector field
            figsize: tuple[float]=(3,3),
            title: str=""):
    "Show the (2D) trajectories evolved according to a dynamical system `f` for `n_pts` random initial points."
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    _f = lambda x: f(x)[0]
    # Random initial points
    pts = center + (np.random.rand(n_pts, 2) * [lims[1] - lims[0], lims[3]- lims[2]] + [lims[0], lims[2]])
    # Draw trajectories
    colors = plt.get_cmap('rainbow')(np.linspace(0,1,n_pts))
    alps = np.logspace(-1, 0, n_steps)
    ax.set_xlim(center[0] + lims[0], center[0] + lims[1])
    ax.set_ylim(center[1] + lims[2], center[1] + lims[3])
    ax.set_title(title)
    for i,p in enumerate(pts): ax.plot(*p, '*', c=colors[i], ms=8, alpha=0.7)  # initial pts
    for t in range(n_steps):
        for i,p in enumerate(pts):
            pts[i] = _f(pts[i])
            ax.plot(*pts[i], '.', c=colors[i], ms=8, alpha=alps[t])
    # Draw vector field
    if showfield:
        _f = lambda x: f(x)[1]
        xs,ys = np.meshgrid(np.linspace(center[0] + lims[0], center[0] + lims[1], 15),
                            np.linspace(center[1] + lims[2], center[1] + lims[3], 15))
        pts = np.vstack([xs.reshape(-1), ys.reshape(-1)])
        vecs = np.apply_along_axis(_f, 0, pts)
        ax.quiver(*pts, *vecs, color='dimgrey')

In [ ]:
#| export
def stream_plot(f: callable, # Dynamical system (x_new, dx) = f(x)
                ax=None,
                center: tuple[float]=(0.,0.),
                lims: tuple[float]=(-1.,1.,-1.,1.), # x,y limits
                cmap: str='inferno',
                figsize: tuple[float]=(3,3),
                title: str=""):
    "Show the (2D) streamlines evolved according to a dynamical system `f`."
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    ax.set_xlim(center[0] + lims[0], center[0] + lims[1])
    ax.set_ylim(center[1] + lims[2], center[1] + lims[3])
    ax.set_title(title)
    _f = lambda x: f(x)[1]
    xs,ys = np.meshgrid(np.linspace(center[0] + lims[0], center[0] + lims[1], 20),
                        np.linspace(center[1] + lims[2], center[1] + lims[3], 20))
    pts = np.vstack([xs.reshape(-1), ys.reshape(-1)])
    vecs = np.apply_along_axis(_f, 0, pts)
    ax.streamplot(*[p.reshape((20,20)) for p in pts], *[v.reshape((20,20)) for v in vecs],
                  color=np.linalg.norm(vecs, axis=0).reshape((20,20)), cmap=cmap, linewidth=.8, arrowsize=.8, arrowstyle='->')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()